Installing the needed libraries and uploading the kaggle auth json file.

In [1]:
! pip install -q kaggle
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


Downloading the dataset form the kaggle API -

In [2]:
! kaggle datasets download -d tboyle10/medicaltranscriptions

 83% 4.00M/4.85M [00:01<00:00, 4.27MB/s]
100% 4.85M/4.85M [00:01<00:00, 3.63MB/s]


Unzip the downloaded dataset and move to the desired target folder using the zipfile library

In [3]:
import zipfile
zf = "/content/medicaltranscriptions.zip"
target_dir = "/content/data"

zfile = zipfile.ZipFile(zf)
zfile.extractall(target_dir)

In [4]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, BatchNormalization, Dropout, Flatten, SpatialDropout1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

drive_dir = "/content/drive/MyDrive/DSAI Project/"

In [15]:
df1 = pd.read_csv(drive_dir + "mtsamples_final.csv", encoding= 'unicode_escape')
# df2 = pd.read_csv("/content/data/mtsamples.csv")
# df3 = pd.read_csv(drive_dir + "back_translated.csv")

# df2 = df2.drop(columns=['Unnamed: 0', 'description', 'sample_name', 'keywords'])
df1 = df1.rename(columns={'medical_specialty': 'Domain', 'transcription': 'Transcriptions'})
# df3 = df3.rename(columns={'medical_specialty': 'Domain', 'backTranslated' : 'Transcriptions'})
# df3 = df3.drop(columns=['transcription','Unnamed: 0', 'french'])

print(df1.shape, df1.columns)
# print(df2.shape, df2.columns)
# print(df3.shape, df3.columns)
df1['Domain'] = df1['Domain'].str.strip().str.capitalize()

(2358, 2) Index(['Domain', 'Transcriptions'], dtype='object')


In [ ]:
label_map = {
    "Orthopedic": "Orthopedic",
    "Radiology": "Radiology",
    "Gastroenterology": "Gastroenterology",
    "Neurology": "Neurology",
    "Urology": "Urology",
    "Ent - otolaryngology": "ENT",
    "Consult - history and phy.": "Consultation",
    "Soap / chart / progress notes": "OPD",
    "Office notes": "Medicine",
    "Physical medicine - rehab": "OPD",
    "Speech - language": "OPD",
    "Surgery": "Surgery",
    "Neurosurgery": "Surgery",
    "General medicine": "Medicine",
    "Dentistry": "Dentistry",
    "Obstetrics / gynecology": "Gynecology",
    "Emergency room reports": "Hospice care",
    "Psychiatry / psychology": "Psychiatry",
    "Dermatology": "Dermatology",
    "Endocrinology": "Endocrinology",
    "Hospice - palliative care": "Hospice care",
    "Hematology - oncology": "Hematology - Oncology",
    "Ophthalmology": "Ophthalmology",
    "Nephrology": "Nephrology",
    "Diets and nutritions": "Diets and Nutritions",
    "Cardiovascular / pulmonary": "Cardiology",
    "Discharge summary": "Others",
    "Pediatrics - neonatal": "Pediatrics",
    "Pain management": "OPD",
    "Podiatry": "OPD",
    "Cosmetic / plastic surgery": "Dermatology",
    "Letters": "Others",
    "Sleep medicine": "Medicine",
    "Bariatrics": "OPD",
    "Ime-qme-work comp etc.": "Consultation",
    "Chiropractic": "Orthopedists",
    "Rheumatology": "Orthopedists",
    "Autopsy": "Others",
    "Lab medicine - pathology": "Medicine",
    "Allergy / immunology": "OPD"
}

In [18]:
# df1['Domain'] = df1['Domain'].str.strip().str.capitalize()
# df2_mod = df2_mod[df2_mod['Domain'] != "Soap / chart / progress notes"]
# df2_mod['Domain'] = df2_mod['Domain'].apply(lambda x : label_map[x])
# df2_mod = df2_mod[df2_mod['Domain'] != "Others"]
# df2_mod = df2_mod[df2_mod['Domain'] != "Consultation"]
# df2_mod = df2_mod[df2_mod['Domain'] != "Medicine"]
# df2_mod = df2_mod[df2_mod['Domain'] != "Surgery"]

# print(df1_mod.shape)
print(df1['Domain'].value_counts())

Orthopedic          659
Radiology           501
Gastroenterology    434
Neurology           416
Urology             348
Name: Domain, dtype: int64


In [19]:
!pip install -q text-preprocessing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 87.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 12.7 MB/s eta 0:00:00


In [21]:
# concat 3 df
df = pd.concat([df1])
df['Domain'] = df['Domain'].str.strip().str.capitalize()
print(df.shape)
print(df['Domain'].value_counts())

(2358, 2)
Orthopedic          659
Radiology           501
Gastroenterology    434
Neurology           416
Urology             348
Name: Domain, dtype: int64


In [22]:
len(df['Domain'].unique())

5

In [23]:
from text_preprocessing import preprocess_text
from text_preprocessing import to_lower, remove_email, remove_url, remove_punctuation, lemmatize_word
from text_preprocessing import remove_number, remove_special_character, normalize_unicode, remove_stopword

# preprocess transcriptions
preprocess_functions = [to_lower, remove_email, remove_number, remove_special_character, remove_url, remove_punctuation, lemmatize_word, remove_stopword]
df['Transcriptions'] = df['Transcriptions'].apply(lambda x: preprocess_text(str(x), preprocess_functions))

# one hot encoding of labels
encoder = OneHotEncoder()
labels = df['Domain']
encoded_labels = encoder.fit_transform(labels.values.reshape(-1, 1))

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
# # train test split
# x_train, x_val, y_train, y_val = train_test_split(df['Transcriptions'].values,encoded_labels, test_size=0.2, random_state=50)

In [24]:
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 81.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 120.8 MB/s eta 0:00:00


In [25]:
!pip install -q evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 74.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 22.9 MB/s eta 0:00:00


In [26]:
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification
from transformers import TextClassificationPipeline

from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import evaluate

In [27]:
df['encoded_text'] = df['Domain'].astype('category').cat.codes

data_texts = df['Transcriptions'].to_list()

data_labels = df['encoded_text'].to_list()

In [28]:
train_texts, val_texts, train_labels, val_labels = train_test_split(data_texts, data_labels, test_size = 0.1, random_state = 50 )


train_texts, test_texts, train_labels, test_labels = train_test_split(train_texts, train_labels, test_size = 0.1, random_state = 50)

In [32]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

train_encodings = tokenizer(train_texts, truncation = True, padding = True  )

val_encodings = tokenizer(val_texts, truncation = True, padding = True )

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))


val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
))

In [33]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=20)

training_args = TFTrainingArguments(
    output_dir='./results',          
    num_train_epochs=10,              
    per_device_train_batch_size=16,  
    per_device_eval_batch_size=16,   
    warmup_steps=500,                
    weight_decay=1e-5,               
    logging_dir='./logs',            
    eval_steps=100,
    evaluation_strategy="epoch"                  
)

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    accuracy = (predictions == labels).mean()
    return {"accuracy": accuracy}

with training_args.strategy.scope():
    trainer_model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels = 20)


trainer = TFTrainer(
    model=trainer_model,                 
    args=training_args,                  
    train_dataset=train_dataset,         
    eval_dataset=val_dataset, 
    compute_metrics= compute_metrics
)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['activation_13', 'vocab_layer_norm', 'vocab_transform', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'pre_classifier', 'dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use i

In [34]:
trainer.train()

In [35]:
trainer.evaluate()

{'eval_loss': 0.4659383773803711, 'eval_accuracy': 0.8}

In [36]:
test_encodings = tokenizer(test_texts, truncation = True, padding = True )
test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_labels
))

trainer.evaluate(eval_dataset=test_dataset)

{'eval_loss': 0.45689892768859863, 'eval_accuracy': 0.7767857142857143}

In [ ]:
trainer_model.save_pretrained(drive_dir + 'models/fine-tune_distil')

tokenizer.save_pretrained(drive_dir + 'models/fine-tune_distil')

('/content/drive/MyDrive/DSAI Project/models/fine-tune_distil/tokenizer_config.json',
 '/content/drive/MyDrive/DSAI Project/models/fine-tune_distil/special_tokens_map.json',
 '/content/drive/MyDrive/DSAI Project/models/fine-tune_distil/vocab.txt',
 '/content/drive/MyDrive/DSAI Project/models/fine-tune_distil/added_tokens.json')

In [ ]:
# tokenizer = DistilBertTokenizer.from_pretrained(drive_dir + 'models/fine-tune_distil')
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

trainer_model = TFDistilBertForSequenceClassification.from_pretrained(drive_dir + 'models/fine-tune_distil')


Some layers from the model checkpoint at /content/drive/MyDrive/DSAI Project/models/fine-tune_distil were not used when initializing TFDistilBertForSequenceClassification: ['dropout_39']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/DSAI Project/models/fine-tune_distil and are newly initialized: ['dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(trainer_model)
tflite_model = converter.convert()

with open(drive_dir + 'models/distilbert.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:
label_mapping = dict(enumerate(df['Domain'].astype('category').cat.categories))
label_mapping

{0: 'Cardiology',
 1: 'Dentistry',
 2: 'Dermatology',
 3: 'Diets and nutritions',
 4: 'Endocrinology',
 5: 'Ent',
 6: 'Gastroenterology',
 7: 'Gynecology',
 8: 'Hematology - oncology',
 9: 'Hospice care',
 10: 'Nephrology',
 11: 'Neurology',
 12: 'Opd',
 13: 'Ophthalmology',
 14: 'Orthopedic',
 15: 'Orthopedists',
 16: 'Pediatrics',
 17: 'Psychiatry',
 18: 'Radiology',
 19: 'Urology'}

In [ ]:
# test_text = test_texts[3]
# print(test_text)
# print(test_labels[3])

test_text = "I was having a severe headache and back pain"
predict_input = tokenizer.encode(
    test_text,
    truncation = True,
    padding = True,
    return_tensors = 'tf'    
)

output = trainer_model(predict_input)[0]

prediction_value = tf.argmax(output, axis = 1).numpy()[0]

prediction_value

11

In [ ]:
predict_input.shape

TensorShape([1, 7])

In [ ]:
model_path = drive_dir + 'models/distilbert.tflite'
model = tf.lite.Interpreter(model_path=model_path)
model.allocate_tensors()

# Define the input and output details
input_details = model.get_input_details()
output_details = model.get_output_details()

sentence = "back pain neck pain"

inputs = tokenizer(sentence, padding=True, truncation=True, return_tensors='tf')

# Get the input and output details of the TFLite model
input_details = model.get_input_details()[0]
output_details = model.get_output_details()[0]
print(inputs['input_ids'].shape)
# Set the input tensor to the tokenized sentence
model.set_tensor(input_details['index'], inputs['input_ids'])

# Run the model inference
model.invoke()

# Get the output tensor and convert it to a numpy array
output_data = model.get_tensor(output_details['index'])
output_data = np.squeeze(output_data)

# Get the label with the highest probability
label_index = np.argmax(output_data)
label = ['Gastroenterology' 'Neurology' 'Orthopedic' 'Radiology' 'Urology'][label_index] 
label

(1, 6)


TypeError: ignored